In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from typing_extensions import TypedDict, List
from typing import Annotated, Optional
from pymongo import MongoClient
from dotenv import load_dotenv
from bson import ObjectId
import importlib
import textwrap
import inspect
import astor
import uuid
import json
import ast
import os
import re

load_dotenv()

In [ ]:
client = MongoClient('mongodb://localhost:27017/')
db = client['agentOrchestratorDB']

In [ ]:
def agent_creation():
    agents_collection = db['agents']

    agent_name = input("Enter Agent Name: ")
    agent_description = input("Enter Agent Description: ")
    agent_prompt = input("Enter Agent Prompt: ")
    
    agent_data = {
        "name": agent_name,
        "description": agent_description,
        "prompt": agent_prompt
    }

    result = agents_collection.insert_one(agent_data)

    print(f"Created agent with object id: {result.inserted_id}")

In [ ]:
def fetch_agents():
    agents_collection = db['agents']
    agents = agents_collection.find()
    available_agents = []
    for i, agent in enumerate(agents, start=1):
        available_agents.append({
            "reference_id": i,
            "agent_id": str(agent.get("_id")),
            "agent_name": agent.get("name"),
            "agent_description": agent.get("description"),
            "agent_prompt": agent.get("prompt"),
            "tools": agent.get("tools")
        })
    
    return available_agents

In [ ]:
def display_agents(agents):
    print("Available Agents:\n")
    for agent in agents:
        print(f"  - Reference ID: {agent['reference_id']}, Agent ID: {agent['agent_id']}, Name: {agent['agent_name']}, Description: {agent['agent_description']}, Prompt: {agent["agent_prompt"][:30]}...{agent["agent_prompt"][-30:]}")

In [ ]:
def create_workflow():
    workdlows_collection = db['workflows']

    agents = fetch_agents()
    display_agents(agents)

    workflow_name = input("Enter Workflow Name: ")
    workflow_description = input("Enter Workflow Description: ")

    selected_ids_input = input("Enter comma-separated Agent Reference IDs to include in the workflow: ")
    try:
        selected_ids = [int(agent_id.strip()) for agent_id in selected_ids_input.split(',')]
    except ValueError:
        print("Invalid input. Please enter only numeric Agent Reference IDs.")
        return

    selected_agents = [agent for agent in agents if agent["reference_id"] in selected_ids]
    if len(selected_agents) != len(selected_ids):
        print("Some Agent IDs were not found. Please try again.")
        return

    workflow_nodes = []
    print("\nNow, define the connections between the selected agents.")
    for agent in selected_agents:
        while True:
            print(f"\nAgent {agent['reference_id']} - {agent['agent_name']}")
            conn_input = input("Enter connected Agent IDs (comma-separated) or press Enter for none: ")

            if conn_input.strip() == "":
                connects = []
                break
            try:
                connects = [int(cid.strip()) for cid in conn_input.split(',')]
            except ValueError:
                print("Invalid input. Please enter only numeric Agent Reference IDs.")
                continue

            if any(cid == agent["reference_id"] for cid in connects):
                print("❌ An agent cannot connect to itself. Please try again.")
                continue

            if all(cid in selected_ids for cid in connects):
                break
            else:
                print("❌ One or more Agent Reference IDs are invalid or not in the selected workflow. Please try again.")

        ref_to_agent = {agent["reference_id"]: agent["agent_id"] for agent in selected_agents}
        connects = [ref_to_agent[cid] for cid in connects if cid in ref_to_agent]

        workflow_nodes.append({
            "agent_id": agent["agent_id"],
            "name": agent["agent_name"],
            "description": agent["agent_description"],
            "connects": connects
        })

    print("\n✅ Workflow Created:")
    print(f"Name: {workflow_name}")
    print(f"Description: {workflow_description}")
    print("Nodes:")
    for node in workflow_nodes:
        print(f"  - Obj ID: {node['agent_id']}, Name: {node['name']}, Connects: {node['connects']}")

    workflow_data = {
        "workflow_name": workflow_name,
        "workflow_description": workflow_description,
        "workflow": workflow_nodes
    }

    result = workdlows_collection.insert_one(workflow_data)

    print(f"Created workflow with object id: {result.inserted_id}")

In [ ]:
def fetch_workflows():
    workflows_collection = db['workflows']
    workflows = workflows_collection.find()

    available_workflows = []
    for i, workflow in enumerate(workflows, start=1):
        available_workflows.append({
            "reference_id": i,
            "workflow_id": str(workflow.get("_id")),
            "workflow_name": workflow.get("workflow_name"),
            "workflow_description": workflow.get("workflow_description"),
            "workflow": workflow.get("workflow")
        })
    
    return available_workflows

In [ ]:
def display_workflows(workflows):
    print("Available Workflows:\n")
    for workflow in workflows:
        print(f"  - Reference ID: {workflow['reference_id']}, Workflow ID: {workflow['workflow_id']}, Name: {workflow['workflow_name']}, Description: {workflow['workflow_description']}")

In [ ]:
def select_workflow():
    workflows = fetch_workflows()
    display_workflows(workflows)

    selected_id_input = input("Enter the Reference ID of a workflow to select: ")
    try:
        selected_id = int(selected_id_input)
    except ValueError:
        print("Invalid input. Please enter only a numeric Reference ID.")
        return

    selected_workflow = next((workflow for workflow in workflows if workflow["reference_id"] == selected_id), None)
    if selected_workflow is None:
        print("\n❌ No workflow found with that Reference ID. Please try again.")
        return

    return selected_workflow

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3
)

In [ ]:
def decide_next_node(state, selected_workflow):
    class DecidingSupervisorResponseFormat(TypedDict):
        next_node: Annotated[str, ..., "Node ID"]
        reasoning: Annotated[str, ..., "Reasoning"]
        instructions: Annotated[str, ..., "Instructions"]
        direct_response: Annotated[Optional[str], None, "Response to user"]
    
    deciding_supervisor_prompt = f"""
        You are a **supervising node/agent** in a directed graph-based team workflow.
        Your job is to oversee and manage task delegation until the user's original task is fully completed.

        Each team member is a node/agent in a graph, connected through the `connects` field.
        When a node finishes its part of the task, you—the supervisor—are informed.
        You must then decide which connected node should handle the next part of the task.
        Delegation is only allowed to nodes listed in the current node’s `connects`.
        Each agent may or may not have a set of tools, the details of which can only be answered by the node itself.

        ---

        ### 🎯 Primary Objective:

        Ensure the **user's task is fully and efficiently completed** by coordinating the workflow through the graph of capable agents.

        ---

        ### 🧠 Your Responsibilities:

        - Understand the **user’s overall task** and the **team structure**.
        - Break down the task into logical subtasks using each member’s `description`.
        - After each node completes a task:
        - Assess progress made so far.
        - Choose the next node from the available `connects`.
        - Provide the selected node with:
            - Relevant task context.
            - A clear description of what they need to do.
            - Any work already completed.

        - Avoid unnecessary or redundant assignments.
        - Make decisions based on:
            - **Skill alignment**
            - **Task continuity**
            - **Workflow logic**

        ---

        ### 🧾 You Will Receive:

        - The full **team structure**, including:
        - `name`, `node` (ID), `description`, and `connects` for each team member.
        - The **original user task**.
        - The **current state**, including:
            - The node that just completed its work.
            - Progress or outputs so far.
            - Chat history and context (always check this before deciding).

        ---

        ### 🧾 Your Output Must Include:

        1. **next_node**: ID of the next node (or `FINISH` if the task is done).
        2. **reasoning**: Justify why this node was chosen, based on the user query, their description and graph connections.
        3. **instructions**: Provide only the specific question or request the user asked.
        4. **direct_response**: A response to the user – only use when you give `next_node` as `FINISH`.

        ---

        ### ⚠️ Constraints:

        - You **can only choose from the current node’s `connects`**.
        - Never allow a node to delegate to itself.
        - If `connects` of previously used agent is empty, or the user just asked a general question (e.g., greetings or “what is your role”), respond with `FINISH` in `next_node`.
        - Once the workflow has started you may not finish until it’s complete (the previous agent has no connects).
        - Don’t assign tasks unless necessary—be efficient and purposeful.
        - Ensure each node receives enough information to pick up the task without confusion.
        - Stick to what user asked for and don't at all respond or ask your agents to respond with stuff user didn't ask for.
        - If the user's message is only requesting **information about agent(s)** (e.g., tools, roles, capabilities), DO NOT interpret it as the start or continuation of a workflow. Only gather and return the requested information.


        **❗ Strict Behavior Rule:**
        - When responding to a user request **about agent(s)**, you must ensure:
            - Agents respond **only to the question asked**, such as their capabilities, tools, or status.
            - Agents must **not begin performing their usual tasks** unless the user explicitly asks them to.
            - If you are querying multiple agents on behalf of the user, provide only the **exact query context** in `instructions`—do not imply task initiation.
            - DO NOT `FINISH` before asking the agent.
        ---

        ### 💬 Special Instruction:

        - You orchestrate the <given workflow> to ensure the user’s task is completed efficiently.
        - If the user asks about **any specific or all agent(s)** and you do **not already know** the answer:
            - You MUST **invoke and query** the relevant agent(s) directly to get accurate information.
            - You MUST prevent any agent from taking action **beyond answering** the specific user query.
            - Never assume or guess an agent's internal capabilities or tools unless explicitly provided.
            - ⚠️ If the user only asked about a **specific agent or their properties**, you need **NOT** follow with the workflow/graph or assign new tasks afterward—only return the requested information.
            - ⚠️ If the user asks about **all agents**, gather the requested information from each one, then return `FINISH` after collecting their responses. Do not initiate or resume the task workflow unless the user explicitly asks for it.

        Then provide the workflow details from the `Given Workflow` section below.

        ---

        ### 🧩 Given Workflow:
        {json.dumps(selected_workflow, indent=2)}
    """


    messages = [SystemMessage(content=deciding_supervisor_prompt)] + state

    supervisor_response = llm.with_structured_output(DecidingSupervisorResponseFormat).invoke(messages)

    return supervisor_response

In [ ]:
def fetch_tool_objects(tool_ids):
    tools_collection = db['tools']
    
    object_ids = [ObjectId(id) for id in tool_ids]
    cursor = tools_collection.find(
        {"_id": {"$in": object_ids}},
        {"_id": 0, "tool_path": 1}
    )
    
    file_path_list = [doc["tool_path"] for doc in cursor]
    tool_list = []
    for path in file_path_list:
        module_name = os.path.splitext(os.path.basename(path))[0]
        spec = importlib.util.spec_from_file_location(module_name, path)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
    
        for name, obj in inspect.getmembers(module):
            if name != 'BaseTool' and inspect.isclass(obj):
                tool_list.append(obj())
                break
    return tool_list

# fetch_tool_objects(["68209fb1c20b476407de799f"])

In [ ]:
def invoke_agent(agent, state):
    if agent['tools'] is None:
        agent['tools'] = []
    tool_ids = [tool_id for tool_id in agent['tools']]
    tools = fetch_tool_objects(tool_ids)
    print(tools)
    worker_agent = create_react_agent(
        model=llm,
        tools=tools,
        prompt=ChatPromptTemplate([
            agent['agent_prompt'], 
            MessagesPlaceholder("messages")
        ]),
        name=agent['agent_name'],
    )
    agent_response = worker_agent.invoke({
        "messages": state
    })
    return agent_response["messages"][-1]

In [ ]:
def invoke_workflow():
    try:
        selected_workflow = select_workflow()
        agent_ids = [node["agent_id"] for node in selected_workflow["workflow"]] + ["FINISH"]
    except Exception:
        return

    user_prompt = input("You: ")
    state = [HumanMessage(content=user_prompt, name="user")]

    supervisor_response = ""
    while True:
        supervisor_response = decide_next_node(state, selected_workflow)
        if supervisor_response["next_node"] == "FINISH":
            if "direct_response" in supervisor_response:
                print(supervisor_response["direct_response"])
            else:
                print("Workflow complete.")
            break
        elif supervisor_response["next_node"] in agent_ids:
            state.append(AIMessage(content=supervisor_response['instructions'], name="supervisor"))
            
            next_agent = next((agent for agent in fetch_agents() if agent["agent_id"] == supervisor_response["next_node"]), None)
            agent_response = invoke_agent(next_agent, state)
            agent_response.pretty_print()
            state.append(agent_response)
    
    return None

In [ ]:
def parse_code(file_content):
    class FunctionCollector(ast.NodeTransformer):
        def __init__(self):
            self.func_defs = []
            self.top_level_calls = []

        def visit_FunctionDef(self, node):
            self.func_defs.append(node)
            return None

        def visit_If(self, node):
            if (isinstance(node.test, ast.Compare)
                and isinstance(node.test.left, ast.Name) and node.test.left.id == '__name__'):
                self.top_level_calls.extend(node.body)
            return None

    tree = ast.parse(file_content)
    collector = FunctionCollector()
    collector.visit(tree)
    
    imports = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Import):
            for alias in node.names:
                imports.append(f"import {alias.name}")
        elif isinstance(node, ast.ImportFrom):
            for alias in node.names:
                imports.append(f"from {node.module} import {alias.name}")

    input_args = set()
    for stmt in collector.top_level_calls:
        for node in ast.walk(stmt):
            if isinstance(node, ast.Call):
                for arg in node.args:
                    if isinstance(arg, ast.Name): 
                        input_args.add(arg.id)
            elif isinstance(node, ast.Assign):
                for target in node.targets:
                    if isinstance(target, ast.Name):
                        input_args.add(target.id)

    input_args = list(input_args)
    if collector.top_level_calls:
        last_stmt = collector.top_level_calls[-1]
        if isinstance(last_stmt, ast.Expr):
            collector.top_level_calls[-1] = ast.Return(value=last_stmt.value)
        elif isinstance(last_stmt, ast.Assign):
            target = last_stmt.targets[0]
            if isinstance(target, ast.Name):
                collector.top_level_calls.append(ast.Return(value=ast.Name(id=target.id, ctx=ast.Load())))
    
    main_func = ast.FunctionDef(
        name="some_function",
        args=ast.arguments(
            args=[ast.arg(arg=name) for name in input_args],
            vararg=None, kwarg=None, defaults=[],
            posonlyargs=[], kwonlyargs=[], kw_defaults=[]
        ),
        body=collector.func_defs + collector.top_level_calls,
        decorator_list=[]
    )

    new_module = ast.Module(body=[main_func], type_ignores=[])

    unified_function = astor.to_source(new_module)

    tree = ast.parse(unified_function)
    params = []

    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            func_name = node.name
            args = [arg.arg for arg in node.args.args]
            if node.args.vararg:
                args.append(node.args.vararg.arg)
            if node.args.kwarg:
                args.append(node.args.kwarg.arg)
            params.append(args)

    function_parameters = params[0]

    return imports, function_parameters, unified_function

In [ ]:
def tool_creation():
    tools_collection = db['tools']
#     print("""
# Sample Code Structure to Successfully Build a Tool
# -------------------------------------------------------------------------------
#     # Valid Imports
#     import os
#     import re

#     def fn1(param1):
#         '''Performs an operation using param1 and calls fn2.'''
#         fn2('value1', 'value2')
#         return 'result_from_fn1'

#     def fn2(param1, param2):
#         '''Processes two parameters and returns a result.'''
#         return 'result_from_fn2'

#     def fn3(number):
#         '''Initiates the process by calling fn1.'''
#         fn1('value_for_fn1')
#         return 'result_from_fn3'

#     number = input("Enter a number: ")
#     fn3(number)  # This is the main function that initiates the workflow.
#     # No further code should be placed below this line.
# -------------------------------------------------------------------------------
# """)

    tool_name = input("Enter Tool Name: ")
    tool_description = input("Enter Tool Description: ")
    file_path = input("Enter Absolute File Path for the code you want to use as a tool: ")

    normalized_path = os.path.normpath(file_path)
    with open(normalized_path, 'r', encoding='utf-8') as file:
        file_content = file.read()

    imports, function_parameters, unified_function = parse_code(file_content)

    details = {
        "imports": imports,
        "tool_name": tool_name,
        "tool_description": tool_description,
        "code": unified_function,
        "function_parameters": function_parameters,
    }

    class FuncParams(TypedDict):
        name: Annotated[str, ..., "Function Parameter Name"]
        type: Annotated[str, ..., "Function Parameter Type"]
        description: Annotated[str, ..., "Function Parameter Description"]
    
    class Router(TypedDict):
        func_params: Annotated[List[FuncParams], ..., "List of Function Parameters"]
    
    system_prompt = """
        Given tool code and other relevant details, your task is to simply assign a valid PYTHON data type and description to the GIVEN function parameters only.
        The code and other given detaiils are for your context only.
        
        Tool Details:
    """ + json.dumps(details)

    llm_respone = llm.with_structured_output(Router).invoke(system_prompt)
    func_params = llm_respone["func_params"]

    tool_name = re.sub(r'[^\w\s]', '', tool_name)
    tool_name = re.sub(r'\s+', ' ', tool_name)

    function_name = tool_name.replace(" ", "_")
    class_name = tool_name.title().replace(" ", "")

    function_params = {item["name"]: item["description"] for item in func_params}
    
    additional_desc = "\nArgs:\n"
    for key, val in function_params.items():
        additional_desc += f"\t{key}: {val}\n"
    tool_description += additional_desc

    import_lines = "\n".join(imports)
    func_params_code = "self"
    if len(func_params) > 0:
        func_params_code += ", " + ", ".join([f"{item['name']}: {item['type']}" for item in func_params])
    
    function_code = re.sub(r'some_function\s*\([^()]*\)', f"_run({func_params_code})", unified_function)
    function_code = re.sub(r'^\s*.*input\(.*\).*$', '', function_code, flags=re.MULTILINE)
    function_code = textwrap.indent(function_code, '\t')

    template = '''
from langchain_core.tools import BaseTool
from typing import Type, Optional
{import_lines}
class {class_name}(BaseTool):
\tname: str = "{function_name}"
\tdescription: str = \"\"\"{tool_description}\"\"\"

{function_code}
'''

    final_code = template.format(
        import_lines=import_lines,
        class_name=class_name,
        function_name=function_name,
        tool_description=tool_description,
        function_code=function_code
    )
    

    tool_dir = os.getenv("TOOLS_DIRECTORY")
    tool_path = os.path.join(tool_dir, f"{tool_name.title().replace(" ", "_")}.py")

    os.makedirs(tool_dir, exist_ok=True)
    
    with open(f"{tool_path}", "w", encoding='utf-8') as file:
        file.write(final_code)
    
    tool_data = {
        "name": tool_name,
        "description": tool_description,
        "tool_path": tool_path
    }
    tools_collection.insert_one(tool_data)
    print("Tool created successfully!")

In [ ]:
def fetch_tools():
    tools_collection = db['tools']
    tools = tools_collection.find()

    available_tools = []
    for i, tool in enumerate(tools, start=1):
        available_tools.append({
            "reference_id": i,
            "tool_id": str(tool.get("_id")),
            "tool_name": tool.get("name"),
            "tool_description": tool.get("description"),
            "tool_path": tool.get("tool_path")
        })
    
    return available_tools

In [ ]:
def map_agents_tools():
    agents = fetch_agents()
    for agent in agents:
        print(f"  - Reference ID: {agent['reference_id']}, Name: {agent['agent_name']}")

    agent_id_input = input("Enter the Reference ID of an agent to select: ")
    try:
        selected_agent_id = int(agent_id_input)
        selected_agent = next(agent for agent in agents if agent["reference_id"] == selected_agent_id)
    except (ValueError, StopIteration):
        print("Invalid input. Please enter a valid numeric Reference ID.")
        return

    tools = fetch_tools()
    print("Available Tools:")
    for tool in tools:
        print(f"  - Reference ID: {tool['reference_id']}, Name: {tool['tool_name']}")

    tool_ids_input = input("Enter comma-separated Reference IDs of tools to select: ")
    try:
        selected_tool_ids = [int(tool_id.strip()) for tool_id in tool_ids_input.split(',')]
        selected_tools = [tool for tool in tools if tool["reference_id"] in selected_tool_ids]
        selected_tool_ids = [selected_tool["tool_id"] for selected_tool in selected_tools]
    except ValueError:
        print("Invalid input. Please enter valid numeric Reference IDs.")
        return

    if len(selected_tools) != len(selected_tool_ids):
        print("Some Tool IDs were not found. Please try again.")
        return

    agents_collection = db['agents']
    agents_collection.update_one(
        {"_id": ObjectId(selected_agent["agent_id"])},
        {"$set": {"tools": selected_tool_ids}}
    )

    print("Tools mapped successfully!")

In [ ]:
invoke_workflow()